# Proyecto de Web Scrapping del Marketplace Yapo.

1. Scrapping de Datos - Yapo

In [2]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#Automcompletar rápido
%config IPCompleter.greedy=True

#Display 6 registros
pd.options.display.min_rows = 6

Con la libreria bs4, abriremos un navegador Firefox, en la pagina de los anuncios de Yapo

In [6]:
browser = webdriver.Firefox() #Cargamos el driver a una variable
myUrl = "https://www.yapo.cl/region-metropolitana/marketplace/computadores-electronica/celulares?orden=date-desc&buscar=iphone&pagina="
browser.get(myUrl) #Abrimos Firefox y entramos a la pagina sin loguearnos.
pageSoup = soup(browser.page_source, "html.parser")

In [10]:
def webscraping(numero):
    pagina = "https://www.yapo.cl/region-metropolitana/marketplace/computadores-electronica/celulares?orden=date-desc&buscar=iphone&pagina="
    pag = pagina+str(numero)
    browser.get(pag) #Abrimos Chrome y entramos a la pagina sin loguearnos.
    time.sleep(3)
    pageSoup = soup(browser.page_source, "html.parser")
    #Comodin para verificar que todo este ok.
    
    #Informacion
    #titulo    
    titulo_h2 = pageSoup.find_all("h2", class_="title m-0 text-truncate pb-2")

    #precio
    precios_span = pageSoup.find_all("span", class_="clp-price d-flex display-price ng-star-inserted")

        #fecha de publicacion
    fecha_p = pageSoup.find_all("p", class_="d-flex m-0 created col ng-star-inserted")

    #hora de publicacion
    hora_p = pageSoup.find_all("p", class_="d-flex m-0 created col ng-star-inserted")

    #lugar
    lugar_span = pageSoup.find_all("span", class_="label align-self-center")

    #dataframes
    df_2 = pd.DataFrame(0, index=range(50), columns=["titulo","precio","fecha_publicacion", "hora_publicacion", "lugar_venta"])
    #Reiniciamos el index para quedar con rangeindex y no con un multiindex
    df_2.reset_index(drop=True, inplace=True)


    #Poblamos titulo.
    df_2["titulo"][:len(titulo_h2)] = titulo_h2
    df_2["titulo"] = df_2["titulo"].astype(str)
    df_2["titulo"]= df_2["titulo"].str.replace('<h2 _ngcontent-serverapp-c2620551781="" class="title m-0 text-truncate pb-2">', '').str.replace('</h2>', '')
    df_2["titulo"]= df_2["titulo"].str.replace('!', '').str.replace('¡', '')

    #Poblamos precio
    df_2["precio"][:len(precios_span)] = precios_span
    df_2["precio"] = df_2["precio"].astype(str)
    df_2["precio"] = (df_2["precio"].str.replace(
            '<span _ngcontent-serverapp-c2620551781="" class="clp-price d-flex display-price ng-star-inserted"><span _ngcontent-serverapp-c2620551781="" class="bracket">(</span>$', '').str.replace(
                '<span _ngcontent-serverapp-c2620551781="" class="bracket">)</span></span>', '').str.replace('.',''))

    #Poblamos la fecha
    df_2["fecha_publicacion"][:len(fecha_p)] = fecha_p
    df_2["fecha_publicacion"]= df_2["fecha_publicacion"].astype(str)
    df_2["fecha_publicacion"] = df_2["fecha_publicacion"].str.replace('<p _ngcontent-serverapp-c2620551781="" class="d-flex m-0 created col ng-star-inserted" title="', '')

    #iteramos para extraer solo la fecha.
    for indice, elemento in enumerate(df_2["fecha_publicacion"]):
        df_2.loc[indice, "fecha_publicacion"] = df_2.loc[indice, "fecha_publicacion"][:10]


    #Poblamos la hora
    df_2["hora_publicacion"][:len(hora_p)] = hora_p
    df_2["hora_publicacion"]= df_2["hora_publicacion"].astype(str)
    df_2["hora_publicacion"] = df_2["hora_publicacion"].str.replace('<p _ngcontent-serverapp-c2620551781="" class="d-flex m-0 created col ng-star-inserted" title="', '')


    #iteramos para extraer solo la hora.
    for indice, elemento in enumerate(df_2["hora_publicacion"]):
        df_2.loc[indice, "hora_publicacion"] = df_2.loc[indice, "hora_publicacion"][12:20]


    #Poblamos el lugar desde donde se publica el producto
    df_2["lugar_venta"][:len(lugar_span)] = lugar_span
    df_2["lugar_venta"] = df_2["lugar_venta"].astype(str)
    df_2["lugar_venta"] = df_2["lugar_venta"].str.replace('<span _ngcontent-serverapp-c2620551781="" class="label align-self-center">', '').str.replace(" <!-- --><!-- -->", "").str.replace("</span>", "")
    
    df_2["pagina"] = numero

    df_2.to_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\02_Apple\WS Datasets_iPhone\ws_iphone_pagina_{numero}.csv", index=False)


In [ ]:
for i in range(1,109):
    if i == 108:
        webscraping(i)
        print(f"Proceso terminado: Ultima pagina {i}")
        break
    else:
        webscraping(i)

In [12]:
#Creamos un nuevo DataFrame
df_final = pd.DataFrame(0, index=range(1), columns=["titulo","precio","fecha_publicacion", "hora_publicacion", "lugar_venta"])

#Poblamos el nuevo DataFrame
for i in range(1,109):
    df_nuevo = pd.read_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\02_Apple\WS Datasets_iPhone\ws_iphone_pagina_{i}.csv")
    # df_nuevo = pd.read_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\01_Samsung\WS Datasets_Samsung\ws_samsung_pagina_{i}.csv")
    df_final = pd.concat([df_final ,df_nuevo],ignore_index=True)
      

In [13]:
#Con Nulos y duplicados - 43901 datos
df_final

,titulo,precio,fecha_publicacion,hora_publicacion,lugar_venta,pagina
0,0,0,0,0,0,NaN
1,Iphone 12 pro 128 silver,600000,24-08-2023,18:20:40,"Ñuñoa, Región Metropolitana",1.0
2,Iphone 11,320000,24-08-2023,17:56:56,"Buin, Región Metropolitana",1.0
...,...,...,...,...,...,...
5398,0,0,0,NaN,0,108.0
5399,0,0,0,NaN,0,108.0
5400,0,0,0,NaN,0,108.0


In [14]:
#Borramos los nulos
df_final.dropna(inplace=True)

In [21]:
#Borramos todas las columnas que tenga 0 en precio
df_final = df_final[df_final["titulo"] != 0]
df_final = df_final[df_final["precio"] != 0]

In [22]:
#Reiniciamos el Dataframe
df_final.reset_index(drop=True, inplace=True)

In [23]:
#sin nulos, duplicados o titulo = 0 - 41244 datos
df_final

,titulo,precio,fecha_publicacion,hora_publicacion,lugar_venta,pagina
0,Iphone 12 pro 128 silver,600000,24-08-2023,18:20:40,"Ñuñoa, Región Metropolitana",1.0
1,Iphone 11,320000,24-08-2023,17:56:56,"Buin, Región Metropolitana",1.0
2,IPhone 7,35000,24-08-2023,16:58:22,"Ñuñoa, Región Metropolitana",1.0
...,...,...,...,...,...,...
4718,Lote laminas de vidrio templado Iphone modelos...,400000,29-07-2022,13:00:42,"Colina, Región Metropolitana",108.0
4719,Instalación de cámara trasera iphone 4s,19990,29-07-2022,12:32:02,"Providencia, Región Metropolitana",108.0
4720,iphone se 128 gb,270000,29-07-2022,10:26:10,"Las Condes, Región Metropolitana",108.0


In [25]:
#Mostramos el Dataframe final
df_final.to_csv(r"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\02_Apple\df_iphone.csv", index=False)